1. Import the CSV File for transactions
2. Read file using pandas with pd.read_csv
3. Next clean data by making a new dataframe and using the .loc function to show only selected columns. 
4. Clean data further by removing transactions that show negative dollar amounts/blanks Transaction ID column. I just created a new variable and removed rows in the column "Transaction ID" based on a condition.
5. Used pandas function to get data type. 
6. Changed Transaction Date column to datetime so I could split it in to year,month, and day columns 
7. Visualize sales per month total, sales per month amount on fees and shipping taken from net and gross difference, avg item sale price, heatmap of states most likely to order and how many sales I got for each day of the week to determine what day of the week most orders occur on. 


In [1]:
import pandas as pd
import numpy as np 
import matplotlib_inline as plt

pd.options.display.max_rows = 374
pd.options.display.max_columns = 374

df = pd.read_csv(r'Transaction-Report-Feb-03-2023-07_27_28-0700-1382246697.csv')
columns = df.loc[:, ['Transaction date' , 'Order number' , 'Buyer username' , 'Ship to province/region/state', 'Ship to zip', 'Ship to country' ,\
     'Net amount' , 'Item ID' , 'Transaction ID' , 'Item title' , 'Gross transaction amount'] ]

print(df)


    Transaction date            Type    Order number  \
0        Jan 1, 2023       Other fee  16-09530-33898   
1        Jan 1, 2023           Order  16-09530-33898   
2       Dec 31, 2022          Payout              --   
3       Dec 29, 2022  Shipping label  15-09518-53603   
4       Dec 29, 2022       Other fee  15-09518-53603   
5       Dec 29, 2022           Order  15-09518-53603   
6       Dec 29, 2022        Transfer              --   
7       Dec 24, 2022  Shipping label  19-09497-45990   
8       Dec 24, 2022        Transfer              --   
9       Dec 24, 2022          Payout              --   
10      Dec 22, 2022       Other fee  19-09497-45990   
11      Dec 22, 2022           Order  19-09497-45990   
12      Dec 13, 2022          Payout              --   
13      Dec 12, 2022  Shipping label  04-09461-54577   
14      Dec 11, 2022       Other fee  04-09461-54577   
15      Dec 11, 2022           Order  04-09461-54577   
16      Dec 11, 2022  Shipping label  11-09456-8

4.  

In [2]:
columns_without_blanks = columns[columns['Transaction ID'] != "--"]


5. Reverse order so that the columns and rows are in order of date Beggining of 2022 to end of 2022 using the loc function in pandas.

In [3]:
cols_by_date = columns_without_blanks.loc[:: -1]
print(cols_by_date)


    Transaction date    Order number         Buyer username  \
373      Jan 1, 2022  16-08077-65484              ruma_2034   
371      Jan 3, 2022  03-08088-91555           jamesth_1138   
367      Jan 7, 2022  23-08098-07154              oxtavea30   
362     Jan 28, 2022  10-08190-70434            samaki_2268   
358     Mar 18, 2022  21-08397-97482             leothelowe   
353     Mar 21, 2022  21-08407-80001               hisan-35   
350     Mar 22, 2022  07-08417-36989                bman437   
345     Mar 25, 2022  02-08429-71369               livinxl2   
341     Mar 29, 2022  11-08445-99388          nintendont420   
338     Mar 29, 2022  04-08447-89453              andba_128   
335     Mar 30, 2022  16-08445-80636           diannarichin   
330      Apr 6, 2022  10-08480-32248             darus-2329   
327      Apr 7, 2022  15-08481-92467             fongpei202   
324      Apr 8, 2022  02-08489-43611         dunbarcolumbus   
322      Apr 8, 2022  18-08485-50599        biddingbuye

In [4]:
cols_by_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 373 to 1
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Transaction date               105 non-null    object 
 1   Order number                   105 non-null    object 
 2   Buyer username                 105 non-null    object 
 3   Ship to province/region/state  105 non-null    object 
 4   Ship to zip                    105 non-null    object 
 5   Ship to country                105 non-null    object 
 6   Net amount                     105 non-null    float64
 7   Item ID                        105 non-null    object 
 8   Transaction ID                 105 non-null    object 
 9   Item title                     105 non-null    object 
 10  Gross transaction amount       105 non-null    float64
dtypes: float64(2), object(9)
memory usage: 9.8+ KB


In [5]:
cols_by_date['Transaction date'] = pd.to_datetime(cols_by_date['Transaction date'])


6. 

In [8]:
#cols_by_date["Year"] = cols_by_date["Transaction date"].dt.strftime("%Y")

cols_by_date["Year"] = cols_by_date["Transaction date"].dt.year
cols_by_date["Month"] = cols_by_date["Transaction date"].dt.month
cols_by_date["Day"] = cols_by_date["Transaction date"].dt.day
print(cols_by_date)

    Transaction date    Order number         Buyer username  \
373       2022-01-01  16-08077-65484              ruma_2034   
371       2022-01-03  03-08088-91555           jamesth_1138   
367       2022-01-07  23-08098-07154              oxtavea30   
362       2022-01-28  10-08190-70434            samaki_2268   
358       2022-03-18  21-08397-97482             leothelowe   
353       2022-03-21  21-08407-80001               hisan-35   
350       2022-03-22  07-08417-36989                bman437   
345       2022-03-25  02-08429-71369               livinxl2   
341       2022-03-29  11-08445-99388          nintendont420   
338       2022-03-29  04-08447-89453              andba_128   
335       2022-03-30  16-08445-80636           diannarichin   
330       2022-04-06  10-08480-32248             darus-2329   
327       2022-04-07  15-08481-92467             fongpei202   
324       2022-04-08  02-08489-43611         dunbarcolumbus   
322       2022-04-08  18-08485-50599        biddingbuye

In [9]:
cols_by_date.head()

,Transaction date,Order number,Buyer username,Ship to province/region/state,Ship to zip,Ship to country,Net amount,Item ID,Transaction ID,Item title,Gross transaction amount,Year,Month,Day
373,2022-01-01,16-08077-65484,ruma_2034,CA,95123-4603,US,13.55,393725788988,1172866574026,The Orange Box: Half-Life 2 & Portal (Xbox 360...,16.00,2022,1,1
371,2022-01-03,03-08088-91555,jamesth_1138,OR,97801-2990,US,13.87,393804431741,1173260955026,Sony PlayStation 2 Electronic Arts Need For Sp...,16.20,2022,1,3
367,2022-01-07,23-08098-07154,oxtavea30,MI,49022-6017,US,14.31,393725779874,1174121149026,"Ninety-Nine Nights N3 (Microsoft Xbox 360, 200...",16.85,2022,1,7
362,2022-01-28,10-08190-70434,samaki_2268,MA,01772-2010,US,17.71,393842113986,1179519447026,Sailor Moon R: Season 2 Part 1 (DVD) *SEALED*,21.25,2022,1,28
358,2022-03-18,21-08397-97482,leothelowe,KY,40601-9500,US,37.77,393992819521,1192060251026,Resident Evil 2 Sony PlayStation 1 PS1 PSOne B...,44.10,2022,3,18
